### Resume Ranking System: A HR assistant program

### Importing Modules

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import requests
import json



### reading resumes.csv file

In [2]:
data = pd.read_csv('resumes.csv')

In [3]:
experience_weight = 0.4
university_weight = 0.3
skills_weight = 0.3
Git_weight = 0.4

In [4]:
data['experience'] = data['experience'].apply(lambda x: x.lower().replace('.', ''))
data['university'] = data['university'].apply(lambda x: x.lower().replace(',', ''))
data['Github_id'] = data['Github_id'].apply(lambda x: x.lower().replace(',', ''))
data['skills'] = data['skills'].apply(lambda x: x.lower().replace(',',''))



In [5]:

def judge_coder(github_id):
    # Get the coder's contributions from GitHub
    url = f"https://api.github.com/users/{github_id}/repos"
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON data
        data = json.loads(response.text)

        # Calculate the number of contributions
        num_contributions = 0
        for repo in data:
            num_contributions += repo["stargazers_count"] + repo["forks"]

        # Calculate the quality of contributions
        quality_score = 0
        for repo in data:
            if repo["stargazers_count"] > 100:
                quality_score += 1
            if repo["forks"] > 50:
                quality_score += 1

        # Calculate the variety of contributions
        variety_score = 0
        for repo in data:
            if repo["language"] in ["Python", "JavaScript", "Java"]:
                variety_score += 1

        # Calculate the experience of the coder
        experience = 0
        for repo in data:
            if repo["created_at"] < "2019-01-01":
                experience += 1

        # Calculate the communication skills of the coder
        communication_score = 0
        for repo in data:
            if repo["open_issues_count"] < 10:
                communication_score += 1
            

        # Calculate the overall score of the coder
        overall_score = (num_contributions + quality_score + variety_score + experience + communication_score) / 5


        # Print the overall score of the coder
        return overall_score
    else:
        print(f"Error: {response.status_code}")
        
Git_scores = data['Github_id'].apply(lambda x: judge_coder(x))





In [6]:
def calculate_experience_score(experience):
    years_of_experience = int(experience.split()[0])
    if years_of_experience <= 2:
        score = 2
    elif years_of_experience <= 5:
        score = 5
    elif years_of_experience <= 10:
        score = 10
    else:
        score = 0.8
    return score

experience_scores = data['experience'].apply(lambda x: calculate_experience_score(x))


In [7]:
def calculate_university_score(university):
    # Assign scores based on university reputation
    if university in ['Stanford University', 'MIT', 'Harvard University']:
        university_score = 3
    elif university in ['University of California, Berkeley', 'University of Cambridge', 'Princeton University']:
        university_score = 2
    else:
        university_score = 1

    return university_score

university_scores = data['university'].apply(lambda x: calculate_university_score(x))


In [8]:
def calculate_skills_score(skills):
    # Define a set of relevant skills
    relevant_skills = {'python', 'java', 'c++', 'sql', 'javascript', 'html', 'css'}

    # Calculate the intersection between the applicant's skills and the relevant skills
    intersection = set(skills).intersection(relevant_skills)

    # Calculate the skills score
    skills_score = len(intersection) / len(relevant_skills)

    return skills_score

skills_scores = data['skills'].apply(lambda x: calculate_skills_score(x))

In [9]:

data['overall_score'] = (
    experience_weight * experience_scores +
    university_weight * university_scores +
    skills_weight * skills_scores +
    Git_weight * Git_scores
)

# Rank resumes
ranked_resumes = data.sort_values(by='overall_score', ascending=False)

ranked_resumes


,name,experience,university,skills,Github_id,overall_score
1,OmkarPathak,2 years,mit,python r sql,omkarpathak,162.54
0,Travis Oliphant,5 years,stanford university,python java c++,teoliphant,21.50
2,Shubham Rajput,1 year,university of california berkeley,java javascript html/css,shubhamrajput768,1.26
